In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import pickle
import os

from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
# pip install -r requirements.txt

In [ ]:
# pip install tensorflow

In [ ]:
# pip install transformers

In [ ]:
# pip install torch

In [ ]:
# Globals for automatic nature
MAKE_DATA = True
ROWS = 100

In [ ]:
# Load data
from data.data import Data

if MAKE_DATA:
    data_raw = Data().get_all_data()
    df = data_raw['MBTI 500']

    df = df[['type', 'posts']]
    df.rename(columns={'posts': 'text'}, inplace=True)
    df = df.sample(n = ROWS, ignore_index = True)

    print(f"MBTI_500 loaded with shape: {df.shape}")


In [ ]:
# Bert Preprocessing
from scripts.BERT_preprocessing import bert_training_preprocessing

df_list = bert_training_preprocessing(df)
df_list[0]

In [ ]:
Xlist = []
for i in range(len(df_list[0]['embeddings'])):
    z = np.array(df_list[0]['embeddings'][i])
    z_pad = np.pad(z, ((0, 200 - z.shape[0]), (0, 0)), 'constant', constant_values=-1000)
    Xlist.append(z_pad)
X_ug = np.vstack(Xlist)

In [ ]:
y = df_list[0].iloc[:,[0]]
X = X_ug.reshape(y.shape[0],-1,768)
print(y.shape, X.shape)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_as_binary = LabelBinarizer()
ylabels = label_as_binary.fit_transform(y)

In [ ]:
#0. Sequential Model
model = Sequential()

model.add(layers.Masking(mask_value=-1000))

#1. Normalization
model.add(layers.Normalization())

#2. SimpleRNN layer, 20 units, tanh
model.add(layers.LSTM(units = 25, activation = 'sigmoid'))


model.add(layers.Dense(512, activation='sigmoid', kernel_regularizer='l1'))

model.add(layers.Dropout(.30))

model.add(layers.Dense(512//2, activation='sigmoid', kernel_regularizer='l1'))
model.add(layers.Dense(512//4, activation='sigmoid', kernel_regularizer='l2'))
model.add(layers.Dense(512//8, activation='sigmoid', kernel_regularizer='l2'))

model.add(layers.Dense(32, activation='relu', kernel_regularizer='l1'))



#4 Dense Layer with Return Layer
model.add(layers.Dense(1, activation = 'sigmoid'))

model.compile(loss = 'mae', 
              optimizer='Adam')
            #   metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 5)

history = model.fit(X, ylabels,
                  validation_split = .2,
                  epochs = 20,
                  batch_size = 32,
                  callbacks = [es])

In [ ]:
model.predict(X[:1])

In [17]:
pd.DataFrame(history.history)

,loss,val_loss
0,65.489227,59.243378
1,54.501732,49.008663
2,44.625706,39.799294
3,35.797600,31.603893
4,28.018009,24.434525
5,21.280476,18.295530
6,15.569308,13.199750
7,10.921538,9.155286
8,7.316830,6.164150
9,4.769752,4.223480


In [ ]:
# Create or Load Models

DIE

from model.DL import *

model_list = initialize_models(metrics='accuracy',
                               loss='binary_crossentropy',
                               vocab_size=768)

In [ ]:
model_list[0].summary()

In [ ]:
# Train Model

train_models(df_list, model_list)

In [ ]:
# Predict Model